This notebook contains code to replicate the experiments mentioned in the Google doc.

The graphs themselves were made by 4o from raw text outputs, so there's no code here for making the actual visualisations.

Warning: A lot of this code is ugly and inefficient. Time constraints! If I were doing a larger scale project I would definitely spend some time making the code nicer.

Big thanks to my bestie o1 who helped with a lot of the code.

Download the data for below experiments here: https://drive.google.com/drive/folders/18t8bGuHln-PVURLCUX4ny0Us7Q3M5XJ2?usp=sharing

# Setup

First load in the models

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from nnsight import NNsight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
small_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
small_tokenizer = AutoTokenizer.from_pretrained(model_name)

small_model = NNsight(small_model)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
large_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
large_tokenizer = AutoTokenizer.from_pretrained(model_name)

large_model = NNsight(large_model)

Now load in the generations

In [ ]:
import os

os.chdir('/home/lfletcher/MATS_app')

In [ ]:
import pickle

with open('data/generations/high_school_gens_large.pkl', 'rb') as f:

    large_gens = pickle.load(f)

with open('data/generations/high_school_gens_small.pkl', 'rb') as f:

    small_gens = pickle.load(f)

Filter the generations to only consider those <2000 tokens and which follow the format of having an answer in \boxed{}.

This code also adds the predicted_label and token position of the predicted label to the generation dict.

In [ ]:
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
small_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

small_filtered_gens = []
for item in small_gens:
    full_text = item['full_generated_text']
    prompt = item['prompt']
    # Estimate tokenized length by splitting on whitespace and punctuation
    tokenized_length = len(small_tokenizer.tokenize(full_text)) - len(small_tokenizer.tokenize(prompt))

    # Check tokenized length condition
    if tokenized_length < 2000:
        # Extract boxed answer using regex patterns
        match = re.search(r'\\boxed{\(?([A-Da-d])\)?}', full_text)
        if match:
            label = match.group(1).upper()  # Normalize to uppercase
            label_index = ord(label) - ord('A')  # Convert A-D to 0-3 index
            answer_str = match.group(0)  # Full match (e.g., \boxed{C})
            char_index = full_text.find(answer_str)  # Character index
            token_index = len(small_tokenizer.tokenize(full_text[:char_index]))  # Convert to token index

            item['predicted_label'] = label_index
            item['answer_token_index'] = token_index  # Store token index of answer
            small_filtered_gens.append(item)

large_filtered_gens = []
for item in large_gens:
    full_text = item['full_generated_text']
    prompt = item['prompt']
    # Estimate tokenized length by splitting on whitespace and punctuation
    tokenized_length = len(small_tokenizer.tokenize(full_text)) - len(small_tokenizer.tokenize(prompt))

    # Check tokenized length condition
    if tokenized_length < 2000:
        # Extract boxed answer using regex patterns
        match = re.search(r'\\boxed{\(?([A-Da-d])\)?}', full_text)
        if match:
            label = match.group(1).upper()  # Normalize to uppercase
            label_index = ord(label) - ord('A')  # Convert A-D to 0-3 index
            answer_str = match.group(0)  # Full match (e.g., \boxed{C})
            char_index = full_text.find(answer_str)  # Character index
            token_index = len(large_tokenizer.tokenize(full_text[:char_index]))  # Convert to token index

            item['predicted_label'] = label_index
            item['answer_token_index'] = token_index  # Store token index of answer
            large_filtered_gens.append(item)



# Experiment 1

First we find the incorrect generations for each of the small and large model

In [ ]:
small_incorrect_gens = [gen for gen in small_filtered_gens if gen['predicted_label'] != gen['correct_label']]
large_incorrect_gens = [gen for gen in large_filtered_gens if gen['predicted_label'] != gen['correct_label']]
                           
print(len(small_incorrect_gens))
print(len(large_incorrect_gens))

Get the pre and post CoT logits for the predicted_answer for the small and large model

This code is monstrously inefficient and not batched. It takes like 10 mins to run on an A100. Sorry!

The resulting lists are dictionaries of the form

    {
        "final_answer": the final answer given by the model after the CoT,
        "post_logits": the logits of A-D after the CoT,
        "pre_logits": the logits of A-D before the CoT,
        "subject": the subject of this datapoint (i.e. geography, math etc.)
    }

In [ ]:
from utils import get_pre_post_cot_logits

small_logits = []
large_logits = []

print("Getting logits for small model")
for i, gen in enumerate(small_incorrect_gens):

    if i % 10 == 0:
        print(f"Getting logits for generation {i}")

    small_logits.append(get_pre_post_cot_logits(gen, small_model, small_tokenizer))

print("\nGetting logits for large model")
for i, gen in enumerate(large_incorrect_gens):

    if i % 10 == 0:
        print(f"Getting logits for generation {i}")
        
    large_logits.append(get_pre_post_cot_logits(gen, large_model, large_tokenizer))

Now that we have these logits, we get a couple of relevant quantities

1) the probability of the final answer pre and post CoT (saved in cur_prob_dict)
2) the probability of all 4 of A-D pre and post CoT (saved in small/large_probs_pre_all and small/large_probs_post_all)

The latter we calculate to check some assumptions that we're relying on (see Google doc)

In [ ]:
import torch.nn.functional as F

small_probs = []

small_probs_pre_all = []
small_probs_post_all = []

for i in range(len(small_incorrect_gens)):

    cur_gen = small_logits[i]
    predict_idx = small_incorrect_gens[i]['predicted_label']

    logits_prompt = cur_gen['pre_logits']
    logits_cot = cur_gen['post_logits']
    
    cur_prob_prompt = F.softmax(logits_prompt)
    cur_prob_cot = F.softmax(logits_cot)

    small_probs_pre_all.append(cur_prob_prompt.tolist())
    small_probs_post_all.append(cur_prob_cot.tolist())

    cur_prob_dict = {"pre_prob":cur_prob_prompt[predict_idx], "post_prob":cur_prob_cot[predict_idx], "subject":cur_gen["subject"]}
    small_probs.append(cur_prob_dict)


large_probs = []

large_probs_pre_all = []
large_probs_post_all = []

for i in range(len(large_incorrect_gens)):

    cur_gen = large_logits[i]
    predict_idx = large_incorrect_gens[i]['predicted_label']

    logits_prompt = cur_gen['pre_logits']
    logits_cot = cur_gen['post_logits']
    
    cur_prob_prompt = F.softmax(logits_prompt)
    cur_prob_cot = F.softmax(logits_cot)

    large_probs_pre_all.append(cur_prob_prompt.tolist())
    large_probs_post_all.append(cur_prob_cot.tolist())

    cur_prob_dict = {"pre_prob":cur_prob_prompt[predict_idx], "post_prob":cur_prob_cot[predict_idx], "subject":cur_gen["subject"]}
    large_probs.append(cur_prob_dict)

Now we get the average probability of the final (post-CoT) answer pre-CoT for each model. This is the main result of this experiment.

In [ ]:
from collections import defaultdict

subject_stats = defaultdict(lambda: {"sum": 0.0, "count": 0})

# Accumulate sums and counts
for entry in small_probs:
    subj = entry["subject"]
    subject_stats[subj]["sum"] += entry["pre_prob"]
    subject_stats[subj]["count"] += 1

# Compute mean and store results
result = []
for subj, stats in subject_stats.items():
    mean_prob = stats["sum"] / stats["count"]
    result.append({
        "subject": subj,
        "mean_pre_prob": mean_prob,
        "count": stats["count"]
    })

print("Results for small model")
for entry in result:
    print(f"Subject: {entry['subject']}, Mean Pre-CoT Probability: {entry['mean_pre_prob']:.4f}, Count: {entry['count']}")


# Accumulate sums and counts
for entry in large_probs:
    subj = entry["subject"]
    subject_stats[subj]["sum"] += entry["pre_prob"]
    subject_stats[subj]["count"] += 1

# Compute mean and store results
result = []
for subj, stats in subject_stats.items():
    mean_prob = stats["sum"] / stats["count"]
    result.append({
        "subject": subj,
        "mean_pre_prob": mean_prob,
        "count": stats["count"]
    })

print("\nResults for large model")
for entry in result:
    print(f"Subject: {entry['subject']}, Mean Pre-CoT Probability: {entry['mean_pre_prob']:.4f}, Count: {entry['count']}")



Finally, we check for one possible alternative explanation for these results, namely that both the pre and post probabilities are biased in the same way 

(i.e., if the model says B 50% of the time both pre and post CoT, then our metric of mean pre-CoT probability will be >50% without post-hoc reasoning being necessary)

In [ ]:
import numpy as np

small_probs_prompt_all = np.array(small_probs_pre_all)
small_probs_cot_all = np.array(small_probs_post_all)

large_probs_prompt_all = np.array(large_probs_pre_all)
large_probs_cot_all = np.array(large_probs_post_all)

print(np.mean(large_probs_prompt_all, axis=0))
print(np.mean(large_probs_cot_all, axis=0))

print(np.mean(small_probs_prompt_all, axis=0))
print(np.mean(small_probs_cot_all, axis=0))

The two distributions are not biased in the same way (although the small model in particular has got a weird bias for B pre-CoT and A post-CoT)

In particular, if pre/post were independent, then for both large and small models the probability of ans_pre = ans_post is approximately 0.25

# Experiment 2

I've moved the code which actually extracts the features to the appendix and just load the features here. This is because the code extracting the features is pretty slow (particularly for the big model). Feel free to skip to the appendix to take a look at that if you like.

So we load the features here

In [ ]:
with open('data/probe_features/small_features.pkl', 'rb') as f:

    small_features = pickle.load(f)

with open('data/probe_features/small_labels.pkl', 'rb') as f:

    small_labels = pickle.load(f)

with open('data/probe_features/large_features.pkl', 'rb') as f:

    large_features = pickle.load(f)

with open('data/probe_features/large_labels.pkl', 'rb') as f:

    large_labels = pickle.load(f)

Now split into test and train sets, along with randomly discarding datapoints until we have balanced labels [balance_classes()] (this is basically the most naive thing you could possibly do but time constraints!)

In [ ]:
from utils import balance_classes
from sklearn.model_selection import train_test_split

def prepare_splits(features, labels):

    bal_features, bal_labels = balance_classes(features, labels)

    features_np = features.numpy()
    labels_np = labels.numpy()

    # Perform an 80–20 split
    train_features_np, test_features_np, \
    train_labels_np, test_labels_np = train_test_split(
        features_np,
        labels_np,
        test_size=0.2,
        random_state=42,
        stratify=labels_np  # Stratify for class balance
    )

    # Convert NumPy arrays back to tensors
    train_features = torch.tensor(train_features_np)
    test_features  = torch.tensor(test_features_np)
    train_labels    = torch.tensor(train_labels_np)
    test_labels     = torch.tensor(test_labels_np)

    return train_features, test_features, train_labels, test_labels

small_train_features, small_test_features, small_train_labels, small_test_labels = prepare_splits(small_features, small_labels)
large_train_features, large_test_features, large_train_labels, large_test_labels = prepare_splits(large_features, large_labels)

Now we train the linear probe 3 times at each position/layer for the small model and print the results

In [ ]:
import torch
import numpy as np
from utils import train_linear_classifier_with_positions


def get_probe_results(train_features, train_labels, test_features, test_labels):

    results = {}

    for layer_idx in range(5):           # Layers 0 through 4
        results[layer_idx] = {}
        
        for pos in range(17):           # Positions 0 through 16
            run_accuracies = []
            
            for run_i in range(3):      # Repeat 3 times
                # Prepare the position labels (replacing the "*2" with "*pos")
                positions_train = torch.ones(len(train_features)).long() * pos
                positions_test  = torch.ones(len(test_features)).long() * pos
                
                # Train the probe
                probe, acc = train_linear_classifier_with_positions(
                    train_features,
                    positions_train,
                    train_labels,
                    test_features,
                    positions_test,
                    test_labels,
                    layer_idx=layer_idx,
                    num_classes=4,    # or whatever your task requires
                    epochs=100,
                    batch_size=32,
                    device='cuda'
                )
                
                # Extract the accuracy (adjust if your function returns the accuracy differently)
                run_accuracies.append(acc)
            
            # Compute mean and std across the 3 runs
            mean_acc = np.mean(run_accuracies)
            std_acc  = np.std(run_accuracies)
            
            # Store in the results structure
            results[layer_idx][pos] = (mean_acc, std_acc)

    return results

small_results = get_probe_results(small_train_features, small_train_labels, small_test_features, small_test_labels)

# Now 'results' contains (mean_acc, std_acc) for each (layer_idx, pos).
# You can do further processing or just print:
for layer_idx in range(5):
    for pos in range(17):
        mean_acc, std_acc = small_results[layer_idx][pos]
        print(f"Layer {layer_idx}, Pos {pos} -> Mean: {mean_acc:.4f}, Std: {std_acc:.4f}")


Now do the same for the large model

In [ ]:

large_results = get_probe_results(large_train_features, large_train_labels, large_test_features, large_test_labels)

for layer_idx in range(5):
    for pos in range(17):
        mean_acc, std_acc = large_results[layer_idx][pos]
        print(f"Layer {layer_idx}, Pos {pos} -> Mean: {mean_acc:.4f}, Std: {std_acc:.4f}")

# Appendix  

Below is the code to generate the features for the linear probe

You need to only load in the relevant model otherwise seems to always OOM? doesn't seem like that should happen

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

del small_model
del large_model

For small model:

In [ ]:
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
small_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

small_filtered_gens = []
for item in small_gens:
    full_text = item['full_generated_text']
    prompt = item['prompt']
    # Estimate tokenized length by splitting on whitespace and punctuation
    tokenized_length = len(small_tokenizer.tokenize(full_text)) - len(small_tokenizer.tokenize(prompt))

    # Check tokenized length condition
    if tokenized_length < 2000:
        # Extract boxed answer using regex patterns
        match = re.search(r'\\boxed{\(?([A-Da-d])\)?}', full_text)
        if match:
            label = match.group(1).upper()  # Normalize to uppercase
            label_index = ord(label) - ord('A')  # Convert A-D to 0-3 index
            answer_str = match.group(0)  # Full match (e.g., \boxed{C})
            char_index = full_text.find(answer_str)  # Character index
            token_index = len(small_tokenizer.tokenize(full_text[:char_index]))  # Convert to token index

            item['predicted_label'] = label_index
            item['answer_token_index'] = token_index  # Store token index of answer
            small_filtered_gens.append(item)

In [ ]:
from torch.utils.data import DataLoader
from utils import TextClassificationDataset, get_probe_features
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from nnsight import NNsight
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
small_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
small_tokenizer = AutoTokenizer.from_pretrained(model_name)

small_model = NNsight(small_model)


dataset = TextClassificationDataset(small_filtered_gens, small_tokenizer, max_length=2000)
dataloader = DataLoader(dataset, batch_size=8)

features, labels, _, _, _ = get_probe_features(small_model, dataloader, hidden_dim=1536, layer_indices=[5,10,15,20,25], max_batches=10000)

with open('data/probe_features/small_features.pkl', 'wb') as f:

    pickle.dump(features, f)

with open('data/probe_features/small_labels.pkl', 'wb') as f:

    pickle.dump(labels, f)

For large model (batch_size=6 is the max that will not cause an OOM error for me)

In [ ]:
from torch.utils.data import DataLoader
from utils import TextClassificationDataset, get_probe_features

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
large_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
large_tokenizer = AutoTokenizer.from_pretrained(model_name)

large_model = NNsight(large_model)

dataset = TextClassificationDataset(large_filtered_gens, small_tokenizer, max_length=2000)
dataloader = DataLoader(dataset, batch_size=6)

features, labels, _, _, _ = get_probe_features(large_model, dataloader, hidden_dim=3584, layer_indices=[5,10,15,20,25], max_batches=10000)

with open('data/probe_features/large_features.pkl', 'wb') as f:

    pickle.dump(features, f)

with open('data/probe_features/large_labels.pkl', 'wb') as f:

    pickle.dump(labels, f)